In [1]:
import os
import sys
import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import math

import import_ipynb
from CustomDataset import ControlsDataset
from Model import ConvNet

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print('using device', device)

importing Jupyter notebook from CustomDataset.ipynb
importing Jupyter notebook from Model.ipynb
using device cpu


In [2]:
torch.cuda.empty_cache()

In [3]:
dataset = ControlsDataset()
dataset.images.set_grayscale(False)
print("Data size", len(dataset.labels))
dataset.make_dataloaders()

Data size 11019
Total training stacks 276
Total validation stacks 69


In [4]:
net = ConvNet(color_channels = 3, outputs = 1, dataset = dataset).to(device)
print("number of parameters: ", sum(p.numel() for p in net.parameters()))

units after conv 512
conv parameters:  168224
fc parameters:  164353
number of parameters:  332577


In [5]:
epochs = 30
optimizer = optim.Adam(net.parameters(), lr=0.0001)
criterion = nn.MSELoss()
net.report_period = 20

net.optimizer = optimizer
net.fit(device,epochs,optimizer,criterion)

0,1	Loss:0.05534	Allocated:naGB	Cached:naGB

0,2	Loss:0.08008	Allocated:naGB	Cached:naGB

0,3	Loss:0.0513	Allocated:naGB	Cached:naGB

0,4	Loss:0.06002	Allocated:naGB	Cached:naGB

0,5	Loss:0.1175	Allocated:naGB	Cached:naGB

0,6	Loss:0.02891	Allocated:naGB	Cached:naGB

0,7	Loss:0.02439	Allocated:naGB	Cached:naGB



KeyboardInterrupt: 

In [ ]:
# End of training Validation Test
#net.load("snapshots/0.727_model.pt")
#total, correct = net.score(device,dataset, "categorical")

total = 0
correct = 0
for i_batch, sampled_batch in enumerate(dataset.validloader):
    images = sampled_batch['image'].to(device).float()
    controls = sampled_batch['control'].to(device).long()
    controls = torch.flatten(controls)
    prediction = net(images)

    maximum = torch.argmax(prediction,dim = 1)
    print(maximum)
    print(controls)
    shared = maximum == controls
    shared = 1 * shared
    correct += int(torch.sum(shared))
    total += len(controls)



print("{:.1f}% classified correctly".format(100*correct/total))

In [ ]:
# Beginning of training Validation Test
net.load("snapshots/3.002_model.pt")
total, correct = net.score(device,dataset, "categorical")
print("{:.1f}% classified correctly".format(100*correct/total))

In [ ]:
torch.cuda.empty_cache()

In [ ]:
#Additional Info when using cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024))